# YOLO-World Batch Testing

Testing YOLO-World on 100 images to evaluate production viability.

In [ ]:
# Step 1: Install ultralytics
!pip install ultralytics

In [ ]:
# Step 2: Set paths
import os
import random
import shutil

PROJECT_ROOT = "c:/Users/Tesla Laptops/Videos/Construction-Site-Safety-PPE-Detection"

# Source: merged dataset images
SOURCE_IMAGES = f"{PROJECT_ROOT}/merged-dataset/train/images"

# Temp folder for test batch
TEST_BATCH_FOLDER = f"{PROJECT_ROOT}/batch_test_images"

# Output folder
OUTPUT_FOLDER = f"{PROJECT_ROOT}/batch_test_results"

# Number of images to test
NUM_IMAGES = 100

In [ ]:
# Step 3: Pick 100 random images
import os
import random
import shutil

# Get all image files
all_images = [f for f in os.listdir(SOURCE_IMAGES) if f.lower().endswith(('.jpg', '.jpeg', '.png'))]
print(f"Total images in dataset: {len(all_images)}")

# Pick random 100
random.seed(42)  # For reproducibility
selected_images = random.sample(all_images, min(NUM_IMAGES, len(all_images)))
print(f"Selected {len(selected_images)} images for testing")

# Copy to test folder
if os.path.exists(TEST_BATCH_FOLDER):
    shutil.rmtree(TEST_BATCH_FOLDER)
os.makedirs(TEST_BATCH_FOLDER)

for img in selected_images:
    src = os.path.join(SOURCE_IMAGES, img)
    dst = os.path.join(TEST_BATCH_FOLDER, img)
    shutil.copy(src, dst)

print(f"Copied {len(selected_images)} images to {TEST_BATCH_FOLDER}")

In [ ]:
# Step 4: Define classes
CLASSES = [
    "hardhat",
    "safety vest",
    "face mask",
    "person",
    "safety cone",
    "chair",
    "machinery",
    "vehicle"
]

In [ ]:
# Step 5: Load YOLO-World
from ultralytics import YOLO

model = YOLO("yolov8x-worldv2.pt")
model.set_classes(CLASSES)

print("Model loaded!")
print(f"Classes: {CLASSES}")

In [ ]:
# Step 6: Run batch inference with timing
import time
import os

os.makedirs(OUTPUT_FOLDER, exist_ok=True)

start_time = time.time()

results = model.predict(
    source=TEST_BATCH_FOLDER,
    save=True,
    save_txt=True,
    save_conf=True,
    project=OUTPUT_FOLDER,
    name="predictions",
    conf=0.25,
    verbose=False
)

end_time = time.time()
total_time = end_time - start_time

print(f"\n{'='*50}")
print(f"BATCH TEST RESULTS")
print(f"{'='*50}")
print(f"Images processed: {len(results)}")
print(f"Total time: {total_time:.2f} seconds")
print(f"Average time per image: {total_time/len(results):.3f} seconds")
print(f"Throughput: {len(results)/total_time:.1f} images/second")

In [ ]:
# Step 7: Analyze detection statistics
import os

labels_path = f"{OUTPUT_FOLDER}/predictions/labels"

class_counts = {}
for cls in CLASSES:
    class_counts[cls] = 0

total_detections = 0
images_with_detections = 0

if os.path.exists(labels_path):
    label_files = os.listdir(labels_path)
    images_with_detections = len(label_files)
    
    for label_file in label_files:
        with open(os.path.join(labels_path, label_file), 'r') as f:
            lines = f.readlines()
            total_detections = total_detections + len(lines)
            
            for line in lines:
                class_id = int(line.split()[0])
                if class_id < len(CLASSES):
                    class_counts[CLASSES[class_id]] = class_counts[CLASSES[class_id]] + 1

print(f"\n{'='*50}")
print(f"DETECTION STATISTICS")
print(f"{'='*50}")
print(f"Images with detections: {images_with_detections}/{NUM_IMAGES}")
print(f"Total detections: {total_detections}")
print(f"Average detections per image: {total_detections/NUM_IMAGES:.1f}")
print(f"\nDetections by class:")
for cls, count in class_counts.items():
    print(f"  {cls}: {count}")

In [ ]:
# Step 8: Production viability summary
print(f"\n{'='*50}")
print(f"PRODUCTION VIABILITY ASSESSMENT")
print(f"{'='*50}")

avg_time = total_time / len(results)

if avg_time < 0.1:
    speed_rating = "EXCELLENT - Real-time capable"
elif avg_time < 0.5:
    speed_rating = "GOOD - Near real-time"
elif avg_time < 2.0:
    speed_rating = "ACCEPTABLE - Batch processing"
else:
    speed_rating = "SLOW - Need GPU acceleration"

detection_rate = images_with_detections / NUM_IMAGES * 100

print(f"Speed Rating: {speed_rating}")
print(f"Detection Rate: {detection_rate:.1f}%")
print(f"\nNote: For production, run on GPU (Colab/Kaggle) for 10-50x speedup")

## Results

Check `batch_test_results/predictions/` for:
- Annotated images with detection boxes
- Label files in YOLO format